In [1]:
from image_processor import scence_understanding

/opt/homebrew/Caskroom/miniconda/base/envs/piramal/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
image = '../data/fe.jpg'
ans = scence_understanding(image)

/opt/homebrew/Caskroom/miniconda/base/envs/piramal/lib/python3.12/site-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [3]:
type(ans)

str

In [4]:
ans

'Hello, I am Sarvam. The explaination for the uploaded photo is this. a blue and white banner'

In [5]:
from image_detection import detect_objects

Using cache found in /Users/akranthreddy/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-11-23 Python-3.12.7 torch-2.6.0.dev20241018 CPU

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients, 4.5 GFLOPs
Adding AutoShape... 
/Users/akranthreddy/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


In [25]:
yolo_string

'person, car, car'

'person, car, car'

In [14]:
from matplotlib import pyplot as plt

In [16]:
plt.imshow(yolo_info['annotated_image'])
plt.show()

In [27]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

def setup_phi2():
    # Initialize model and tokenizer
    model_name = "microsoft/phi-2"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,  # Use float16 for efficiency
        device_map="auto"  # Automatically handle device placement
    )
    return model, tokenizer

def generate_summary(model, tokenizer, text, max_length=128):
    # Create an effective prompt template for summarization
    prompt_template = """Below is some text that needs to be summarized concisely. 
    Focus on the key information and main points.

    Text: {input_text}

    Provide a brief summary of the above text:"""
    
    full_prompt = prompt_template.format(input_text=text)
    
    # Tokenize and generate
    inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            inputs.input_ids,
            max_length=max_length,
            num_return_sequences=1,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
        )
    
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract only the generated summary part
    return summary.split("Provide a brief summary of the above text:")[-1].strip()


In [28]:
model, tokenizer = setup_phi2()
    
# Your specific text
text = "Hello, I am Sarvam. The explaination for the uploaded photo is this. a blue and white bannerperson, car, car"

summary = generate_summary(model, tokenizer, text)
print("Generated Summary:", summary)


KeyboardInterrupt: 

In [31]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load model and tokenizer
model_name = "t5-small"  # Change to "facebook/bart-large-cnn" or "sshleifer/distilbart-cnn-12-6" for BART models
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Define the text to summarize
text = """
i want you to rewrite the following text into easily readable format.
scene: Hello, I am Sarvam. The explaination for the uploaded photo is this. 
number of objects: a blue and white bannerperson, car, car
"""

# Prepare the text for the model
inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)

# Generate summary
outputs = model.generate(inputs, max_length=50, min_length=10, length_penalty=2.0, num_beams=4, early_stopping=True)

# Decode and print the summary
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Summary:", summary)


Summary: i want you to rewrite the following text into easily readable format. i want you to rewrite the following text into easily readable format.


In [34]:
import requests

# Define the URL for the Ollama API on your host
OLLAMA_URL = "http://127.0.0.1:11434/api/chat"

# The payload for interacting with a model
payload = {
    "model": "lama2",  # Replace with the model you want to use
    "messages": [{"role": "user", "content": "What is Ollama?"}]
}


response = requests.post(OLLAMA_URL, json=payload)
response.raise_for_status()  # Raise an exception for HTTP errors

# Parse and display the response
result = response.json()
print("Ollama Response:", result.get("response", "No response"))



HTTPError: 404 Client Error: Not Found for url: http://127.0.0.1:11434/api/chat

In [3]:
scene = "Hello, I am Sarvam. The explaination for the uploaded photo is this. a blue and white banner"
objects_scene = "person, car, car"

In [4]:
from summarizer import summarize

In [5]:
text = summarize(scene, objects_scene)

In [6]:
from translate import translate

In [7]:
ans = translate(text)
import json



In [11]:
text

'Here\'s the rewritten text:\n\n"Hello, I\'m Sarvam. This is an explanation of the image you\'ve uploaded: A blue and white banner is prominently featured on screen, accompanied by a person, a car, and various objects that add depth to the scene."'

In [17]:
import requests

url = "https://api.sarvam.ai/text-to-speech"

payload = {
    "inputs": [text], 
    "target_language_code": "hi-IN",
    "speaker": "meera",
    "pitch": 0,
    "pace": 1.65,
    "loudness": 1.5,
    "speech_sample_rate": 8000,
    "enable_preprocessing": True,
    "model": "bulbul:v1"
}
headers = {"Content-Type": "application/json", "api-subscription-key":"a4d97b92-9fbd-4346-8870-178df79b1515"}

response = requests.request("POST", url, json=payload, headers=headers)

print(response)

<Response [200]>


In [24]:
len(response.json()['audios'][0])

17152

In [25]:
from gtts import gTTS

# Your text in BCP-47 format
text = response.json()['audios'][0]
language = "en"  # Replace with the appropriate BCP-47 language code

# Convert text to speech
tts = gTTS(text=text, lang=language)

# Save the speech to an MP3 file
tts.save("output.mp3")

print("MP3 file has been created successfully.")

KeyboardInterrupt: 